# Deployment

## FastAPI

We strongly recommend deploying Marvin's components with FastAPI. 

### Example: Deploying a *Declarative* API Gateway

Here's how you can deploy a declarative API gateway in a few lines of code.

In [ ]:
from fastapi import FastAPI
from marvin import ai_fn, ai_model
from pydantic import BaseModel
import uvicorn
import asyncio

app = FastAPI()


@ai_fn
def generate_fruits(n: int) -> list[str]:
    """Generates a list of `n` fruits"""


@ai_fn
def generate_vegetables(n: int, color: str) -> list[str]:
    """Generates a list of `n` vegetables of color `color`"""


@ai_model
class Person(BaseModel):
    first_name: str
    last_name: str


app.add_api_route("/generate_fruits", generate_fruits)
app.add_api_route("/generate_vegetables", generate_vegetables)
app.add_api_route("/person/extract", Person.route())

If you want to serve the previous example from, say, a Jupyter Notebook for local testing, you can also include:

In [ ]:
# ... from above
# If you want to run an API from a Jupyter Notebook.

config = uvicorn.Config(app)
server = uvicorn.Server(config)
await server.serve()

# Then navigate to localhost:8000/docs

### Example: Deploying an AI Application

REST APIs are, by their very nature, stateless. This means that holding the `state` of an application needs to be shared by the user (by managing the state in their browser) and/or the server (via a database). 

#### Having the end-user manage state. 
For this example, we'll forgo persisting the state to a database (which is otherwise recommended) and show how you can manage state. 

In [ ]:
from marvin import AIApplication

from datetime import datetime
from pydantic import BaseModel
from fastapi import FastAPI

app = FastAPI()


class ToDo(BaseModel):
    """A to-do item."""

    title: str
    description: str
    due_date: datetime = None
    done: bool = False


class ToDoState(BaseModel):
    """The state of the to-do tracker."""

    todos: list[ToDo] = []


class ToDoResponse(BaseModel):
    """The response from the to-do tracker."""

    content: str
    state: ToDoState


@app.get("/")
def run(update: str, state: ToDoState = ToDoState()) -> ToDoResponse:
    description = (
        "A simple to-do tracker. Users will give instructions "
        "to add, remove, and update their to-dos."
    )

    # We'll initialize the application, passing in the state and description.
    todo = AIApplication(state=state, description=description)

    # We'll issue the update to the application.
    response = todo(update)

    # We'll return the response, along with the updated state.
    return ToDoResponse(content=response.content, state=todo.state)

Every time you call /?update=, you'll get a response from the to-do tracker acknowledging the update, along
with the current state of the tracker. In this implementation you'll need to keep the state on the client side
and pass it in with every request.

We'll use FastAPI's test client to send a mock request.

In [ ]:
from fastapi.testclient import TestClient

response = TestClient(app).get(
    url="/",
    params={"update": "Remind me I have an appointment with Craig tomorrow at 9AM"},
)

response.json()

# {
#   "content": "You have an appointment with Craig tomorrow at 9AM.",
#   "state": {
#     "todos": [
#       {
#         "title": "Appointment with Craig",
#         "description": "Meeting with Craig tomorrow at 9AM",
#         "due_date": "2023-07-19T09:00:00",
#         "done": false
#       }
#     ]
#   }
# }